## 1. 환경 설정

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

## 2. LangChain RAG 구현

### 2.1 문서 로더 (Document Loaders)

In [ ]:
# 텍스트 파일 목록 가져오기
txt_files = glob(os.path.join('data', '*_KR.txt'))

print(txt_files)

In [ ]:
# 첫 번째 텍스트 파일 내용 확인
from langchain_community.document_loaders import TextLoader

loader = TextLoader(txt_files[0], encoding='utf-8')
data = loader.load()
len(data)

In [ ]:
# data type 확인
type(data)

In [ ]:
# page_content를 출력
print(data[0].page_content)

In [ ]:
# metadata를 출력
print(data[0].metadata)

In [ ]:
# Load all files from a directory
from tqdm import tqdm

data = []

for text_file in tqdm(txt_files):
    loader = TextLoader(text_file, encoding='utf-8')
    data += loader.load()

len(data)

In [ ]:
print(data[0].page_content)

In [ ]:
print(data[1].page_content)

In [ ]:
type(data[0])

### 2.2 텍스트 분할기 (Text Splitters)

In [ ]:
# 각 문서의 글자수를 계산
char_count = [len(doc.page_content) for doc in data]

print(char_count)

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=250,        # 청크 크기  
    chunk_overlap=50,      # 청크 중 중복되는 부분 크기
    separator='\n\n',      # 청크를 나눌 때 사용할 구분자
)

texts = text_splitter.split_documents(data)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

- CharacterTextSplitter는 단순히 문자 수로만 텍스트를 자르지 않습니다. 
- 기본적으로 문장이나 단락의 경계를 존중하려고 합니다. 
- 따라서 문장이나 단락의 끝부분에서 지정된 chunk_size를 약간 초과할 수 있습니다.

In [ ]:
print("첫 번째 청크의 내용:", texts[0].page_content)

In [ ]:
print("두 번째 청크의 내용:", texts[1].page_content)

In [ ]:
# 겹치는 부분을 출력
print("첫 번째 청크의 최종 50글자:", texts[0].page_content[-50:])
print("두 번째 청크의 처음 50글자:", texts[1].page_content[:50])

In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=250,        # 청크 크기  
    chunk_overlap=50,      # 청크 중 중복되는 부분 크기
    separator='',          # 청크를 나눌 때 사용할 구분자
)

texts = text_splitter.split_documents(data)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

In [ ]:
# 겹치는 부분을 출력
print("첫 번째 청크의 최종 50글자:", texts[0].page_content[-50:])
print("두 번째 청크의 처음 50글자:", texts[1].page_content[:50])

### 2.3 임베딩 모델 (Embeddings)

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름을 지정 가능 
)
sample_text = "테슬라 창업자는 누구인가요?"
vector = embeddings.embed_query(sample_text)
print(f"임베딩 벡터의 차원: {len(vector)}")

### 2.4 벡터 저장소 (Vector Stores)

In [ ]:
texts[0]

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=texts, 
    embedding=embeddings,
    collection_name="chroma_test", 
    persist_directory="./chroma_db",
    )
print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

In [ ]:
# 문서 검색
query = "테슬라 창업자는 누구인가요?"
result = vectorstore.similarity_search(query)

print(f"검색 결과의 길이: {len(result)}")
print(f"검색 결과의 첫 번째 문서: {result[0].page_content[:100]}...")

### 2.5 검색기 (Retrievers)
   - 질의에 관련된 문서를 검색하는 컴포넌트

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

relevant_docs = retriever.invoke(query)
print(f"검색된 관련 문서 수: {len(relevant_docs)}")
print(f"첫 번째 관련 문서 내용 미리보기: {relevant_docs[0].page_content[:100]}...")

### 2.6 언어 모델 (LLMs)

In [ ]:
# Context를 제공하지 않고 답변 생성 (Hallucination 발생 위험)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",  # 사용할 모델 이름을 지정 가능
    temperature=0,        # 답변의 창의성을 조절
    max_tokens=100,       # 생성할 최대 토큰 수
    )

response = llm.invoke("테슬라 창업자는 누구인가요?")
print(f"LLM 응답: {response}")

In [ ]:
# Context를 제공하고 답변 생성

query_with_context = f"""{relevant_docs[0].page_content}\n\n위 내용에 근거하여 다음 질문에 답변하세요. \n\n{query}"""
print(query_with_context)

In [ ]:
response = llm.invoke(query_with_context)
print(f"LLM 응답: {response.content}")

### 2.7 전체 RAG 파이프라인 구성

In [28]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



# LLM 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 프롬프트 - 옵션 1 (허브 사용)
# https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
# retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

# 프롬프트 - 옵션 2 (직접 작성)
retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면, 
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")


# 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke({"input": query})

In [ ]:
# 결과 출력
print(response)

In [ ]:
# 응답 객체의 키 확인
response.keys()

In [ ]:
response["answer"]

`[추가 설명] create_stuff_documents_chain`
- 검색된 문서들을 하나의 컨텍스트로 결합하고, 이를 바탕으로 질문에 답변하는 체인을 생성

In [ ]:

from langchain_core.documents import Document

stuff_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

documents = [
    Document(page_content="LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다."),
    Document(page_content="LangChain은 파이썬과 자바스크립트를 지원합니다.")
]
response = stuff_chain.invoke({
    "input": "LangChain은 어떤 언어를 지원하나요?",
    "context": documents
})
print(response)

`[추가 설명] create_retrieval_chain`
- 질문에 관련된 문서를 검색하고, 검색된 문서를 바탕으로 답변을 생성하는 전체 RAG 파이프라인을 구축

In [ ]:
# 검색 체인 생성
retrieval_chain = create_retrieval_chain(vectorstore.as_retriever(), stuff_chain)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = retrieval_chain.invoke({"input": query})

# 결과 출력
print(response)

## 3. Gradio 챗봇

In [ ]:
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke({"input": message})
    return response["answer"]


# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

In [ ]:
# Graiio 종료
demo.close()